# Explora aquí


Se recomienda utilizar este cuaderno con fines de exploración.

In [2]:
# Install required packages (only run once)
%pip install beautifulsoup4 requests matplotlib seaborn

# Import necessary libraries
import os
import time
import sqlite3
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup




  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/104.0 kB ? eta -:--:--
     -------------- ---------------------- 41.0/104.0 kB 960.0 kB/s eta 0:00:01
     ----------------------- --------------- 61.4/104.0 kB 1.1 MB/s eta 0:00:01
     ----------------------------- ------- 81.9/104.0 kB 573.4 kB/s eta 0:00:01
     ------------------------------------ 104.0/104.0 kB 544.9 kB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     -------------------------- ------------- 41.0/60.8 kB 2.0 MB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 648.8 kB/s eta 0:00:00
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/186.0 kB


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
